> ## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import Pipeline
import plotly.express as px

# custom modules
import sys
sys.path.append('../')
from scripts.dataframe import load_data as ld;

> ## Load Dataset

In [5]:
# Load training data
data_paths = {
    'train' : '../docs/train_data.txt',
    'test':'../docs/test_data.txt',
    'test_solution':'../docs/test_data_solution.txt'
}
train_data = ld(data_paths['train'])
train_data.head()

,id,title,genre,plot
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his doc...
1,2,Cupid (1997),thriller,A brother and sister with a past incestuous re...
2,3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fiel...
3,4,The Secret Sin (1915),drama,To help their unemployed father make ends meet...
4,5,The Unrecovered (2007),drama,The film's title refers not only to the un-rec...


> ## EDA

In [10]:
train_data.describe()

,id,title,genre,plot
count,54214,54214,54214,54214
unique,54214,54214,27,54086
top,54214,Nature's Fury: Storm of the Century (2006),drama,Grammy - music award of the American academy o...
freq,1,1,13613,12
